# **TPS - Oct 2021**

## **XGBoost & LightGBM & CatBoost Stacking**

### Thank you for visiting my notebook :)
### This notebook is for beginner like me **who wants to study stacking ensemble!**

#### **Stacking Ensemble** is a nice technique for forwarding you score.
#### As you can see below image, Stacking Ensemble needs some models for classification and meta-model for final prediction!

#### Here's what you need to do.
**Step1. Make your train, test data for training & prediction (Preprocessing)**

**Step2. Select some models for making stacking datasets!! (Train models and Making Datasets)**

**Step3. Select final model for meta-model!**

**Step4. With your meta-model, Train & Predict with stacking datasets ;)**

![Stacking Ensemble](http://rasbt.github.io/mlxtend/user_guide/classifier/StackingClassifier_files/stackingclassification_overview.png)

# **Import Library**

In [2]:
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# **Load Data**

In [3]:
# train_data = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')

clx = pd.read_csv('../input/tpsoct2021/stacking_train_clx.csv')
clx2_B1 = pd.read_csv('../input/tpsoct2021/stacking_train_clx2_B1.csv')
clx2_B2 = pd.read_csv('../input/tpsoct2021/stacking_train_clx2_B2.csv')
clx2_B3 = pd.read_csv('../input/tpsoct2021/stacking_train_clx2_B3.csv')
clx3_fc1 = pd.read_csv('../input/tpsoct2021/stacking_train_clx3_fc1.csv')
clx3_fc2 = pd.read_csv('../input/tpsoct2021/stacking_train_clx3_fc2.csv')
nn1 = pd.read_csv('../input/tpsoct2021-nn/oof_NN1.csv')
nn2 = pd.read_csv('../input/tpsoct2021-nn/oof_NN2.csv')
nn3 = pd.read_csv('../input/tpsoct2021-nn/oof_NN3.csv')
target = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv', usecols=['target'])
# test_data = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
clx_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx.csv')
clx2_B1_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx2_B1.csv')
clx2_B2_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx2_B2.csv')
clx2_B3_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx2_B3.csv')
clx3_fc1_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx3_fc1.csv')
clx3_fc2_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx3_fc2.csv')
nn1_test = pd.read_csv('../input/tpsoct2021-nn/submission_NN1.csv',usecols=['target'])
nn2_test = pd.read_csv('../input/tpsoct2021-nn/submission_NN2.csv',usecols=['target'])
nn3_test = pd.read_csv('../input/tpsoct2021-nn/submission_NN3.csv',usecols=['target'])




print("Data imported")

Data imported


In [4]:
target.head()

,target
0,1
1,1
2,1
3,1
4,1


In [5]:
train_data = pd.concat([clx, clx2_B1, clx2_B2, clx2_B3, clx3_fc1, clx3_fc2, nn1, nn2, nn3], axis=1)
del clx, clx2_B1, clx2_B2, clx2_B3, clx3_fc1, clx3_fc2, nn1, nn2, nn3
gc.collect()

0

In [6]:
train_data.head()

,cat1,lgbm1,xgb1,cat2_B1,lgbm2_B1,xgb2_B1,cat2_B2,lgbm2_B2,xgb2_B2,cat2_B3,...,xgb2_B3,cat3_fc1,lgbm3_fc1,xgb3_fc1,cat3_fc2,lgbm3_fc2,xgb3_fc2,NN1,NN2,NN3
0,0.636516,0.659099,0.654883,0.768058,0.755764,0.764824,0.554211,0.605594,0.626031,0.794097,...,0.778191,0.558595,0.603123,0.607533,0.603311,0.616937,0.593090,0.465069,0.438382,0.440719
1,0.162521,0.186299,0.212586,0.366222,0.394592,0.389743,0.140930,0.162203,0.161350,0.422175,...,0.378377,0.151585,0.169611,0.177332,0.211078,0.192040,0.191485,0.471271,0.354400,0.345115
2,0.844496,0.852618,0.838314,0.484043,0.553811,0.554177,0.825709,0.858642,0.858205,0.517371,...,0.555654,0.862844,0.862994,0.856937,0.869971,0.874981,0.873232,0.850723,0.865030,0.831066
3,0.439135,0.527092,0.547408,0.722231,0.743720,0.747319,0.507045,0.553381,0.576203,0.732503,...,0.738712,0.587314,0.565700,0.620190,0.463408,0.492358,0.472813,0.489519,0.581348,0.532480
4,0.842151,0.830427,0.850084,0.522609,0.468266,0.474799,0.823246,0.815355,0.827911,0.481159,...,0.531830,0.818003,0.794510,0.800700,0.877110,0.831257,0.860441,0.878740,0.878620,0.910610


In [7]:
nn1_test.rename(columns={'target':'NN1'},inplace=True)
nn2_test.rename(columns={'target':'NN2'},inplace=True)
nn3_test.rename(columns={'target':'NN3'},inplace=True)

In [8]:
test_data = pd.concat([clx_test, clx2_B1_test, clx2_B2_test, clx2_B3_test, clx3_fc1_test, clx3_fc2_test,nn1_test, nn2_test, nn3_test], axis=1)
del clx_test, clx2_B1_test, clx2_B2_test, clx2_B3_test, clx3_fc1_test, clx3_fc2_test,nn1_test, nn2_test, nn3_test
gc.collect()

46

In [9]:
test_data.head()

,cat1,lgbm1,xgb1,cat2_B1,lgbm2_B1,xgb2_B1,cat2_B2,lgbm2_B2,xgb2_B2,cat2_B3,...,xgb2_B3,cat3_fc1,lgbm3_fc1,xgb3_fc1,cat3_fc2,lgbm3_fc2,xgb3_fc2,NN1,NN2,NN3
0,0.743567,0.725492,0.773843,0.379822,0.382807,0.382681,0.724124,0.736472,0.751203,0.390824,...,0.385558,0.722077,0.726661,0.744193,0.644519,0.412591,0.372536,0.604470,0.636643,0.613098
1,0.257177,0.247080,0.236851,0.466345,0.463944,0.487700,0.266524,0.252382,0.234636,0.455833,...,0.427886,0.265878,0.258234,0.253721,0.294631,0.529993,0.502947,0.348088,0.360540,0.370865
2,0.905104,0.918153,0.908728,0.592032,0.603186,0.587983,0.938882,0.946836,0.943005,0.531833,...,0.535434,0.936350,0.951365,0.946484,0.897739,0.790795,0.784437,0.878787,0.912274,0.913245
3,0.845935,0.836258,0.824549,0.532147,0.575965,0.556360,0.825365,0.824783,0.815317,0.550670,...,0.580966,0.822680,0.827817,0.817863,0.776154,0.611829,0.566786,0.743707,0.712590,0.733588
4,0.262387,0.277417,0.249826,0.492827,0.471721,0.471770,0.276785,0.286744,0.261271,0.466032,...,0.457980,0.277331,0.274110,0.266399,0.331941,0.525243,0.527473,0.320587,0.327888,0.325372


|||||||||||||||||||||||||||||||||||||||||||||||||||||

||||||||||||||||||||||||||||||||||||

In [10]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [11]:
train = reduce_memory_usage(train_data)
test = reduce_memory_usage(test_data)
y = reduce_memory_usage(target)

Mem. usage decreased to 40.05 Mb (75.0% reduction)
Mem. usage decreased to 20.03 Mb (75.0% reduction)
Mem. usage decreased to 0.95 Mb (87.5% reduction)


## **Checking Missing Values**

In [ ]:
print(train.isna().sum(), test.isna().sum())

# Feature Generation

In [ ]:
# train['std'] = train.std(axis=1)
# # train['min'] = train.min(axis=1)
# # train['max'] = train.max(axis=1)

# test['std'] = test.std(axis=1)
# # test['min'] = test.min(axis=1)
# # test['max'] = test.max(axis=1)

In [ ]:
# y = train['target']
# train.drop(columns = ['id', 'target'], inplace = True)
# test.drop(columns = 'id', inplace = True)

# **Modeling**

### **Stacking Data Loader**

In [ ]:
# def Stacking_Data_Loader(model, model_name, train, y, test, fold):
#     '''
#     Put your train, test datasets and fold value!
#     This function returns train, test datasets for stacking ensemble :)
#     '''

#     stk = StratifiedKFold(n_splits = fold, random_state = 42, shuffle = True)
    
#     # Declaration Pred Datasets
#     train_fold_pred = np.zeros((train.shape[0], 1))
#     test_pred = np.zeros((test.shape[0], fold))
    
#     for counter, (train_index, valid_index) in enumerate(stk.split(train, y)):
#         x_train, y_train = train.iloc[train_index], y[train_index]
#         x_valid, y_valid = train.iloc[valid_index], y[valid_index]

#         print('------------ Fold', counter+1, 'Start! ------------')
#         if model_name == 'cat':
#             model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)])
#         elif model_name == 'xgb':
#             model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric = 'auc', verbose = 500, early_stopping_rounds = 200)
#         elif model_name == 'lgbm':
#             model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric = 'auc', verbose = 500, early_stopping_rounds = 200)
#         print('------------ Fold', counter+1, 'Done! ------------')
        
#         train_fold_pred[valid_index, :] = model.predict_proba(x_valid)[:, 1].reshape(-1, 1)
#         test_pred[:, counter] = model.predict_proba(test)[:, 1]
        
#         del x_train, y_train, x_valid, y_valid
#         gc.collect()
        
#     test_pred_mean = np.mean(test_pred, axis = 1).reshape(-1, 1)
    
#     del test_pred
#     gc.collect()
    
#     print('Done!')
    
#     return train_fold_pred, test_pred_mean

### **Modeling**

#### Model's HyperParameters
* LGBM Param : https://www.kaggle.com/hiro5299834/tps-oct-2021-single-lightgbm
* Cat Param : https://www.kaggle.com/ranjeetshrivastav/tps-oct-21-catboost
* xgb Param : https://www.kaggle.com/rahulchauhan3j/tps-oct-2021-xgboost-pipeline-with-optuna#Model-Fit-and-Submission

Thanks for Sharing!

In [ ]:
# lgb_params = {
#     'objective': 'binary',
#     'n_estimators': 20000,
#     'random_state': 42,
#     'learning_rate': 8e-3,
#     'subsample': 0.6,
#     'subsample_freq': 1,
#     'colsample_bytree': 0.4,
#     'reg_alpha': 10.0,
#     'reg_lambda': 1e-1,
#     'min_child_weight': 256,
#     'min_child_samples': 20,
# #     'device': 'cpu',
#     'device': 'gpu',
# }


# xgb_params = {'n_estimators': 10000,
#                'learning_rate': 0.03689407512484644,
#                'max_depth': 8,
#                'colsample_bytree': 0.3723914688159835,
#                'subsample': 0.780714581166012,
#                'eval_metric': 'auc',
#                'use_label_encoder': False,
#                'gamma': 0,
#                'reg_lambda': 50.0,
# #                'tree_method': 'hist',
#                 'tree_method': 'gpu_hist',
#                'gpu_id': 0,
# #                'predictor': 'cpu_predictor'
#                 'predictor': 'gpu_predictor',
#                'random_state': 42}

# cat_params = {'iterations': 17298,
#                'learning_rate': 0.03429054860458741,
#                'reg_lambda': 0.3242286463210283,
#                'subsample': 0.9433911589913944,
#                'random_strength': 22.4849972385133,
#                'depth': 8,
#                'min_data_in_leaf': 4,
#                'leaf_estimation_iterations': 8,
#                'task_type':"GPU",
#                'bootstrap_type':'Poisson',
#                'verbose' : 500,
#                'early_stopping_rounds' : 200,
#                'eval_metric' : 'AUC'}


In [ ]:
# lgbm = LGBMClassifier(**lgb_params)

# xgb = XGBClassifier(**xgb_params)

# cat = CatBoostClassifier(**cat_params)

### **Stacking**

* Making train, test prediction array!
* Concat 3 arrays in 1 dataset

In [ ]:
# cat_train, cat_test = Stacking_Data_Loader(cat, 'cat', train, y, test, 5)
# del cat
# gc.collect()

# lgbm_train, lgbm_test = Stacking_Data_Loader(lgbm, 'lgbm', train, y, test, 5)
# del lgbm
# gc.collect()

# xgb_train, xgb_test = Stacking_Data_Loader(xgb, 'xgb', train, y, test, 5)
# del xgb
# gc.collect()

### **Final Stacking Datasets!**

In [ ]:
# stack_x_train = np.concatenate((cat_train, lgbm_train, xgb_train), axis = 1)
# stack_x_test = np.concatenate((cat_test, lgbm_test, xgb_test), axis = 1)
# # stack_x_train = np.concatenate((lgbm_train, xgb_train), axis = 1)
# # stack_x_test = np.concatenate(( lgbm_test, xgb_test), axis = 1)
# del cat_train, lgbm_train, xgb_train, cat_test, lgbm_test, xgb_test
# gc.collect()

# stack_x_train

In [ ]:
# stack_x_train.shape

### **Save stack_x_train**

In [ ]:
# cols = ['cat1','lgbm1', 'xgb1']
# df_stack = pd.DataFrame(stack_x_train,columns = cols)
# df_stack.to_csv('stacking_train_clx.csv', index = 0)
# df_stack.head()

In [ ]:
# cols = ['cat1','lgbm1', 'xgb1']
# df_test = pd.DataFrame(stack_x_test,columns = cols)
# df_test.to_csv('stacking_train_clx.csv', index = 0)
# df_test.head()

## Level 2 Training & Optuna

In [ ]:
# params = {'device_type': 'cpu',
#  'n_estimators': 10000,
#  'learning_rate': 0.10932948761384702,
#  'num_leaves': 780,
#  'max_depth': 7,
#  'min_data_in_leaf': 9200,
#  'lambda_l1': 20,
#  'lambda_l2': 5,
#  'min_gain_to_split': 8.857618869765401,
#  'bagging_fraction': 0.8,
#  'bagging_freq': 1,
#  'feature_fraction': 0.8}

# overfit = {'device': 'cpu',
# #               'gpu_platform_id': 0,
# #               'gpu_device_id': 0,
#               'num_threads': -1,
#          'objective': 'binary','metric' : 'auc',
#     'n_estimators': 100000,
#     'random_state': 69,
#     'learning_rate': 3e-3,
#          }

In [15]:
def lgbm_fit(params, train, test, y):
    stk = StratifiedKFold(n_splits = 5)

    test_pred_lo = 0
    fold = 1
    total_auc = 0

    stack_x_train = np.array(train)
    stack_x_test = np.array(test)
    y_n = y.target
    train_fold_pred = np.zeros((train.shape[0], 1))
    
    for train_index, valid_index in stk.split(stack_x_train, y_n):
        x_train, y_train = stack_x_train[train_index], y_n[train_index]
        x_valid, y_valid = stack_x_train[valid_index], y_n[valid_index]

    #     model = RandomForestClassifier(random_state=32)
        model = LGBMClassifier(**params)
        model.fit(x_train, y_train
                  ,eval_set=[(x_valid,y_valid)],
                 early_stopping_rounds=200,
                verbose=0
                 )
        train_fold_pred[valid_index, :] = model.predict_proba(x_valid)[:, 1].reshape(-1, 1)
        valid_pred_lo = model.predict_proba(x_valid)[:, 1]
        test_pred_lo += model.predict_proba(stack_x_test)[:, 1]
        auc = roc_auc_score(y_valid, valid_pred_lo)
        total_auc += auc / 5
        print('Fold', fold, 'AUC :', auc)
        fold += 1

    print('Total AUC score :', total_auc)
    return train_fold_pred, test_pred_lo

In [18]:
params = {
    'objective': 'binary',
    'metric' : 'auc',
    'random_state': 32,
}

In [19]:
best_cols = ['NN3', 'cat1', 'NN1', 'xgb2_B3', 'cat2_B3', 'cat2_B1', 'lgbm1']
train_pred0, test_pred0= lgbm_fit(params, train[best_cols], test[best_cols], y)

Fold 1 AUC : 0.8571221933162716
Fold 2 AUC : 0.8597632960512852
Fold 3 AUC : 0.8597831688699837
Fold 4 AUC : 0.855301813053476
Fold 5 AUC : 0.85548216607317
Total AUC score : 0.8574905274728372


In [40]:
def train_model_optuna(trial, X_train, X_valid, y_train, y_valid):
    """
    A function to train a model using different hyperparamerters combinations provided by Optuna. 
    Loss of validation data predictions is returned to estimate hyperparameters effectiveness.
    """
    
        
    #A set of hyperparameters to optimize by optuna
    lgbm_params = {
                    "objective": trial.suggest_categorical("objective", ['binary']),
                    "boosting_type": trial.suggest_categorical("boosting_type", ['gbdt']),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 256),
                    "max_depth": trial.suggest_int("max_depth", 1, 6),
#                     "max_depth": trial.suggest_categorical("max_depth", [8]),
                    "learning_rate": trial.suggest_float("learning_rate", 0.15, 1, step=0.01),
                    "n_estimators": trial.suggest_categorical("n_estimators", [40000]),        
                    "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 100.0, step=0.1),
                    "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 100.0, step=0.1),
                    "random_state": trial.suggest_categorical("random_state", [42]),
                    "bagging_seed": trial.suggest_categorical("bagging_seed", [42]),
                    "feature_fraction_seed": trial.suggest_categorical("feature_fraction_seed", [42]), 
                    "n_jobs": trial.suggest_categorical("n_jobs", [4]), 
                    "subsample": trial.suggest_float("subsample", 0.1, 1, step=0.01),
                    "subsample_freq": trial.suggest_int("subsample_freq", 1, 7),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1, step=0.01),
#                     "colsample_bytree": trial.suggest_categorical("colsample_bytree", [1]),
#                     "device_type": trial.suggest_categorical("device_type", ["GPU"]),
                    'min_child_samples': trial.suggest_int('min_child_samples', 5, 300),
                    'min_child_weight': trial.suggest_int('min_child_weight', 256, 512),
        
                    }



    # Model loading and training
    model = LGBMClassifier(**lgbm_params)
    model.fit(X_train, y_train,
              eval_set=[(X_valid, y_valid)],
              eval_metric="auc",
              early_stopping_rounds=100,
              verbose=False)
    
    print(f"Number of boosting rounds: {model.best_iteration_}")
    oof = model.predict_proba(X_valid)[:, 1]
    
    return roc_auc_score(y_valid, oof)

In [42]:
import optuna

In [43]:
%%time
# Splitting data into train and valid folds using target bins for stratification
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state=42, stratify=y)

# Setting optuna verbosity to show only warning messages
# If the line is uncommeted each iteration results will be shown
# optuna.logging.set_verbosity(optuna.logging.WARNING)

time_limit = 3600 * 4

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: train_model_optuna(trial, X_train, X_valid,
                                                    y_train, y_valid),
#                n_trials = 2
               timeout=time_limit
              )

# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2021-11-03 06:40:45,853] A new study created in memory with name: no-name-34df9f6a-5419-494d-97d8-9b92a34de37f
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Number of boosting rounds: 46


[I 2021-11-03 06:40:56,994] Trial 0 finished with value: 0.8572733652585094 and parameters: {'objective': 'binary', 'boosting_type': 'gbdt', 'num_leaves': 214, 'max_depth': 5, 'learning_rate': 0.52, 'n_estimators': 40000, 'reg_alpha': 58.1, 'reg_lambda': 31.400000000000002, 'random_state': 42, 'bagging_seed': 42, 'feature_fraction_seed': 42, 'n_jobs': 4, 'subsample': 0.67, 'subsample_freq': 1, 'colsample_bytree': 0.29, 'min_child_samples': 91, 'min_child_weight': 323}. Best is trial 0 with value: 0.8572733652585094.
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2021-11-03 06:41:04,024] Trial 1 finished with value: 0.856900665407836 and parameters: {'objective': 'binary', 'boosting_type': 'gbdt', 'num_leaves': 92, 'max_depth': 2, 'learning_rate': 0.93, 'n_estimators': 40000, 'reg_alpha

Number of boosting rounds: 50


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

Number of boosting rounds: 53


[I 2021-11-03 10:40:25,731] Trial 672 finished with value: 0.8575972044632096 and parameters: {'objective': 'binary', 'boosting_type': 'gbdt', 'num_leaves': 31, 'max_depth': 4, 'learning_rate': 0.2, 'n_estimators': 40000, 'reg_alpha': 54.900000000000006, 'reg_lambda': 23.400000000000002, 'random_state': 42, 'bagging_seed': 42, 'feature_fraction_seed': 42, 'n_jobs': 4, 'subsample': 0.92, 'subsample_freq': 6, 'colsample_bytree': 0.8500000000000001, 'min_child_samples': 190, 'min_child_weight': 305}. Best is trial 649 with value: 0.8576376537512683.
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Number of boosting rounds: 56


[I 2021-11-03 10:40:38,223] Trial 673 finished with value: 0.8575360967057134 and parameters: {'objective': 'binary', 'boosting_type': 'gbdt', 'num_leaves': 13, 'max_depth': 5, 'learning_rate': 0.21, 'n_estimators': 40000, 'reg_alpha': 68.8, 'reg_lambda': 17.000000000000004, 'random_state': 42, 'bagging_seed': 42, 'feature_fraction_seed': 42, 'n_jobs': 4, 'subsample': 0.9, 'subsample_freq': 6, 'colsample_bytree': 0.6100000000000001, 'min_child_samples': 179, 'min_child_weight': 315}. Best is trial 649 with value: 0.8576376537512683.
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2021-11-03 10:40:51,430] Trial 674 finished with value: 0.857605261970791 and parameters: {'objective': 'binary', 'boosting_type': 'gbdt', 'num_leaves': 28, 'max_depth': 5, 'learning_rate': 0.22, 'n_estimators'

Number of boosting rounds: 32
Number of finished trials: 675
Best trial parameters: {'objective': 'binary', 'boosting_type': 'gbdt', 'num_leaves': 66, 'max_depth': 5, 'learning_rate': 0.15, 'n_estimators': 40000, 'reg_alpha': 79.8, 'reg_lambda': 13.1, 'random_state': 42, 'bagging_seed': 42, 'feature_fraction_seed': 42, 'n_jobs': 4, 'subsample': 0.9, 'subsample_freq': 6, 'colsample_bytree': 0.92, 'min_child_samples': 208, 'min_child_weight': 312}
Best score: 0.8576376537512683
CPU times: user 15h 30min 12s, sys: 1min 2s, total: 15h 31min 14s
Wall time: 4h 12s


In [46]:
best_cols = ['NN3', 'cat1', 'NN1', 'xgb2_B3', 'cat2_B3', 'cat2_B1', 'lgbm1']
best_params =  study.best_params
train_pred, test_pred= lgbm_fit(best_params, train[best_cols], test[best_cols], y)

Fold 1 AUC : 0.8570969981425656
Fold 2 AUC : 0.8597676695554003
Fold 3 AUC : 0.8597377486772477
Fold 4 AUC : 0.8552838191365454
Fold 5 AUC : 0.8554769970183066
Total AUC score : 0.8574726465060132


# **Submission!**

## Blending Level 2 results

In [48]:
ver="n002"

In [49]:
cols = [f'model1_{ver}']
# stack_x_train = np.stack((train_pred_array[0], train_pred_array[1], train_pred_array[2]), 1)
# stack_x_test = np.stack((test_pred_array[0], test_pred_array[1], test_pred_array[2]), 1)
df_train_pred = pd.DataFrame(train_pred, columns = cols)
df_test_pred = pd.DataFrame(test_pred, columns = cols)

df_train_pred.to_csv(f'lgbm_train{ver}.csv', index=0)
df_test_pred.to_csv(f'lgbm_test{ver}.csv', index=0)
sub = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')
sub['target'] = test_pred
sub.to_csv('sub_lgbm.csv', index = 0)

sub.head()

,id,target
0,1000000,3.480241
1,1000001,1.424085
2,1000002,4.674285
3,1000003,4.065621
4,1000004,1.521898


# Done!


## If you think this notebook is helpful for you, Please do not forget upvote!